In [ ]:
import os
import pandas as pd
from datetime import timedelta

folder_path = r"/content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0"

def seconds_to_hms(seconds):
    return str(timedelta(seconds=seconds))

def hms_to_seconds(hms):
    if hms is None or hms == 'None':
        return 0
    h, m, s = map(int, hms.split(':'))
    return h * 3600 + m * 60 + s

def parse_summary_file(file_path):
    data = []
    file_name = None
    start_time = 'NA'
    end_time = 'NA'

    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines_iter = iter(lines)
        for line in lines_iter:
            if line.startswith("File Name:"):
                file_name = line.split(": ")[1].strip()
            elif line.startswith("File Start Time:"):
                start_time = line.split(": ")[1].strip()
            elif line.startswith("File End Time:"):
                end_time = line.split(": ")[1].strip()
            elif line.startswith("Number of Seizures in File:"):
                num_seizures = int(line.split(": ")[1].strip())
                seizures = []
                seizure_durations = []
                if num_seizures > 0:
                    for i in range(num_seizures):
                        seizure_start = None
                        seizure_end = None
                        try:
                            line = next(lines_iter).strip()
                            if f"Seizure {i+1} Start Time:" in line or "Seizure Start Time:" in line:
                                seizure_start = int(line.split(": ")[1].strip().split()[0])
                            line = next(lines_iter).strip()
                            if f"Seizure {i+1} End Time:" in line or "Seizure End Time:" in line:
                                seizure_end = int(line.split(": ")[1].strip().split()[0])

                            seizure_start_hms = seconds_to_hms(seizure_start)
                            seizure_end_hms = seconds_to_hms(seizure_end)
                            seizure_duration = seizure_end - seizure_start
                            seizure_duration_hms = seconds_to_hms(seizure_duration)

                            seizures.append(f"({seizure_start_hms},{seizure_end_hms})")
                            seizure_durations.append(seizure_duration)
                        except Exception as e:
                            print(f"Error parsing seizures for file {file_name}: {e}")
                    seizures_str = ",".join(seizures)
                    seizure_durations_str = ",".join([seconds_to_hms(sd) for sd in seizure_durations])
                    average_seizure_duration = seconds_to_hms(sum(seizure_durations) // num_seizures)
                    if file_name:
                        data.append([file_name, start_time, end_time, num_seizures, seizures_str, seizure_durations_str, average_seizure_duration])
                    else:
                        print(f"Warning: Missing file name in file {file_path}")
                else:
                    if file_name:
                        data.append([file_name, start_time, end_time, num_seizures, None, None, None])
                    else:
                        print(f"Warning: Missing file name in file {file_path}")

    return data

all_data = []
for i in range(1, 25):
    file_num = str(i).zfill(2)
    file_name = f'chb{file_num}/chb{file_num}-summary.txt'
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        print(f"Parsing file: {file_path}")
        file_data = parse_summary_file(file_path)
        if file_data:
            print(f"Data parsed from file {file_name}: {file_data}")
        all_data.extend(file_data)
    else:
        print(f"File does not exist: {file_path}")

if all_data:
    df = pd.DataFrame(all_data, columns=['File Name', 'Start Time', 'End Time', 'Number of Seizures', 'Seizure Times', 'Seizure Durations', 'Average Seizure Duration'])

    # Save the DataFrame to CSV
    o1_csv_path = 'o1.csv'
    df.to_csv(o1_csv_path, index=False)
    print(f"CSV file saved to {o1_csv_path}")

    # Calculate total average seizure duration for all files
    # Convert 'Average Seizure Duration' to seconds and calculate the mean
    df['Average Seizure Duration Seconds'] = df['Average Seizure Duration'].apply(lambda x: hms_to_seconds(x))
    total_avg_seizure_duration_seconds = df['Average Seizure Duration Seconds'].sum() / df[df['Number of Seizures'] > 0].shape[0]
    total_avg_seizure_duration = seconds_to_hms(total_avg_seizure_duration_seconds)
    print(f"Total average seizure duration for all files: {total_avg_seizure_duration}")
else:
    print("No data parsed. CSV file not created.")


Parsing file: /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01-summary.txt
Data parsed from file chb01/chb01-summary.txt: [['chb01_01.edf', '11:42:54', '12:42:54', 0, None, None, None], ['chb01_02.edf', '12:42:57', '13:42:57', 0, None, None, None], ['chb01_03.edf', '13:43:04', '14:43:04', 1, '(0:49:56,0:50:36)', '0:00:40', '0:00:40'], ['chb01_04.edf', '14:43:12', '15:43:12', 1, '(0:24:27,0:24:54)', '0:00:27', '0:00:27'], ['chb01_05.edf', '15:43:19', '16:43:19', 0, None, None, None], ['chb01_06.edf', '16:43:26', '17:43:26', 0, None, None, None], ['chb01_07.edf', '17:43:33', '18:43:33', 0, None, None, None], ['chb01_08.edf', '18:43:40', '19:43:40', 0, None, None, None], ['chb01_09.edf', '19:43:56', '20:43:56', 0, None, None, None], ['chb01_10.edf', '20:44:07', '21:44:07', 0, None, None, None], ['chb01_11.edf', '21:44:14', '22:44:14', 0, None, None, None], ['chb01_12.edf', '22:44:22', '23:44:22', 0, None, None, None], ['chb01_13.edf', '23:44:29', '24:44:29',

In [ ]:
pip install pyedflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 19.3 MB/s eta 0:00:00


In [ ]:
#file extenstions and naming change once a single block of code runs so re running raises an file not found error

In [ ]:
import os
import pyedflib
import numpy as np
import pandas as pd

def extract_segments(data_path, edf_file, start_time, end_time):
    file_path = os.path.join(data_path, edf_file)

    try:
        f = pyedflib.EdfReader(file_path)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None, None, None

    sfreq = f.getSampleFrequency(0)
    start_sample = int(start_time * sfreq)
    end_sample = int(end_time * sfreq)

    segment_data = []
    for i in range(f.signals_in_file):
        signal = f.readSignal(i)[start_sample:end_sample]
        segment_data.append(signal)

    f.close()

    segment_data = np.array(segment_data)

    return segment_data, sfreq, f

def save_as_edf(segment_data, sfreq, f, output_file):
    n_channels = segment_data.shape[0]
    signal_headers = f.getSignalHeaders()

    edf_writer = pyedflib.EdfWriter(output_file, n_channels, file_type=pyedflib.FILETYPE_EDFPLUS)

    for i in range(n_channels):
        min_value = segment_data[i].min()
        max_value = segment_data[i].max()

        if min_value == max_value:
            max_value += 1.0

        signal_headers[i]['physical_min'] = min_value
        signal_headers[i]['physical_max'] = max_value
        signal_headers[i]['digital_min'] = -32768
        signal_headers[i]['digital_max'] = 32767
        signal_headers[i]['sample_frequency'] = sfreq

    edf_writer.setSignalHeaders(signal_headers)
    edf_writer.setPatientCode(f.getPatientCode())
    edf_writer.setPatientName(f.getPatientName())
    edf_writer.setStartdatetime(f.getStartdatetime())

    edf_writer.writeSamples(segment_data)
    edf_writer.close()

csv_file_path = r"/content/o1.csv"
csv_data = pd.read_csv(csv_file_path)

base_data_path = r"/content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0"
seizure_output_dir = r"/content/drive/MyDrive/EEG Signal Classification/SEIZURE_EXTRACTION"
non_seizure_output_dir = r"/content/drive/MyDrive/EEG Signal Classification/NON_SEIZURE_EXTRACTION"
os.makedirs(seizure_output_dir, exist_ok=True)
os.makedirs(non_seizure_output_dir, exist_ok=True)

for i in range(1, 25):
    folder_name = f'chb{str(i).zfill(2)}'
    folder_path = os.path.join(base_data_path, folder_name)

    for index, row in csv_data.iterrows():
        edf_file = row['File Name']
        if edf_file.startswith(folder_name):
            num_seizures = row['Number of Seizures']
            seizure_times = row['Seizure Times']

            if num_seizures > 0 and pd.notna(seizure_times):
                seizure_times = seizure_times.strip('()').replace(' ', '').split('),(')
                seizure_intervals = []
                for seizure_time in seizure_times:
                    times = seizure_time.split(',')
                    if len(times) == 2:
                        start_time_str, end_time_str = times

                        start_time_parts = start_time_str.split(':')
                        end_time_parts = end_time_str.split(':')

                        seizure_start_time = int(start_time_parts[0]) * 3600 + int(start_time_parts[1]) * 60 + int(start_time_parts[2])
                        seizure_end_time = int(end_time_parts[0]) * 3600 + int(end_time_parts[1]) * 60 + int(end_time_parts[2])

                        seizure_intervals.append((seizure_start_time, seizure_end_time))

                for j, (seizure_start_time, seizure_end_time) in enumerate(seizure_intervals):
                    print(f"Processing seizure {j + 1} from {seizure_start_time} to {seizure_end_time} in file {edf_file}")

                    seizure_data, sfreq, f = extract_segments(folder_path, edf_file, seizure_start_time, seizure_end_time)

                    if seizure_data is not None:
                        output_file_seizure = os.path.join(seizure_output_dir, f"{edf_file.replace('.edf', '')}_seizure{j + 1}.edf")
                        save_as_edf(seizure_data, sfreq, f, output_file_seizure)
                        print(f"Seizure segment saved as {output_file_seizure}")
                    else:
                        print(f"Failed to extract seizure segment from {edf_file}")


                for j, (seizure_start_time, seizure_end_time) in enumerate(seizure_intervals):
                   if seizure_start_time >= 75:
                    non_seizure_start_time = max(0, seizure_start_time - 75)
                    non_seizure_end_time = seizure_start_time
                   else:
                      non_seizure_start_time = seizure_end_time
                      non_seizure_end_time = min(seizure_end_time + 75, 3600)

                   if all(non_seizure_start_time >= end or non_seizure_end_time <= start for start, end in seizure_intervals):
                    print(f"Processing non-seizure segment {j + 1} from {non_seizure_start_time} to {non_seizure_end_time} in file {edf_file}")

                    non_seizure_data, sfreq, f = extract_segments(folder_path, edf_file, non_seizure_start_time, non_seizure_end_time)

                    if non_seizure_data is not None:
                        output_file_non_seizure = os.path.join(non_seizure_output_dir, f"{edf_file.replace('.edf', '')}_non_seizure{j + 1}.edf")
                        save_as_edf(non_seizure_data, sfreq, f, output_file_non_seizure)
                        print(f"Non-seizure segment saved as {output_file_non_seizure}")
                    else:
                     print(f"Failed to extract non-seizure segment from {edf_file}")
                   else:
                      print(f"Non-seizure segment {j + 1} overlaps with seizure in file {edf_file}, skipping.")

print("Processing complete.")


Processing seizure 1 from 2996 to 3036 in file chb01_03.edf
Error: /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_03.edf: can not open file, no such file or directory
Failed to extract seizure segment from chb01_03.edf
Processing non-seizure segment 1 from 2921 to 2996 in file chb01_03.edf
Error: /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_03.edf: can not open file, no such file or directory
Failed to extract non-seizure segment from chb01_03.edf
Processing seizure 1 from 1467 to 1494 in file chb01_04.edf
Error: /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_04.edf: can not open file, no such file or directory
Failed to extract seizure segment from chb01_04.edf
Processing non-seizure segment 1 from 1392 to 1467 in file chb01_04.edf
Error: /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0/chb01/chb01_04.edf: can not open file, no such file or directory
Failed to extrac

/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (FP1-F7) is -319.1208791208791, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to -319.120, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical maximum for channel 0 (FP1-F7) is 275.16483516483515, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to 275.1648, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (F7-T7) is -766.3003663003664, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to -766.300, some loss of precision is to be expected
  warnings.warn(

Seizure segment saved as /content/drive/MyDrive/EEG Signal Classification/SEIZURE_EXTRACTION/chb10_12_seizure1.edf
Processing non-seizure segment 1 from 6238 to 6313 in file chb10_12.edf


/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (FP1-F7) is -411.13553113553115, which has 19 chars, however, EDF+ can only save 8 chars, will be truncated to -411.135, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical maximum for channel 0 (FP1-F7) is 523.0769230769231, which has 17 chars, however, EDF+ can only save 8 chars, will be truncated to 523.0769, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (F7-T7) is -382.4175824175824, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to -382.417, some loss of precision is to be expected
  warnings.warn(

Non-seizure segment saved as /content/drive/MyDrive/EEG Signal Classification/NON_SEIZURE_EXTRACTION/chb10_12_non_seizure1.edf
Processing seizure 1 from 6888 to 6958 in file chb10_20.edf


...

/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (FP1-F7) is -202.68620268620268, which has 19 chars, however, EDF+ can only save 8 chars, will be truncated to -202.686, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical maximum for channel 0 (FP1-F7) is 342.36874236874235, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to 342.3687, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (F7-T7) is -217.33821733821733, which has 19 chars, however, EDF+ can only save 8 chars, will be truncated to -217.338, some loss of precision is to be expected
  warnings.war

Non-seizure segment saved as /content/drive/MyDrive/EEG Signal Classification/NON_SEIZURE_EXTRACTION/chb15_22_non_seizure1.edf
Processing seizure 1 from 876 to 1066 in file chb15_28.edf


/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (FP1-F7) is -231.990231990232, which has 17 chars, however, EDF+ can only save 8 chars, will be truncated to -231.990, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:131: UserWarning: Physical maximum for channel 0 (FP1-F7) is 403.9072039072039, which has 17 chars, however, EDF+ can only save 8 chars, will be truncated to 403.9072, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.10/dist-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 1 (F7-T7) is -229.05982905982904, which has 19 chars, however, EDF+ can only save 8 chars, will be truncated to -229.059, some loss of precision is to be expected
  warnings.warn('

Seizure segment saved as /content/drive/MyDrive/EEG Signal Classification/SEIZURE_EXTRACTION/chb15_28_seizure1.edf
Processing non-seizure segment 1 from 801 to 876 in file chb15_28.edf


KeyboardInterrupt: 

In [ ]:
import os
import pyedflib
import numpy as np


def extract_channels(input_folder, output_folder, channels):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".edf"):
            input_filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, filename)

            f = pyedflib.EdfReader(input_filepath)
            available_channels = f.getSignalLabels()
            print(f"Available channels in {filename}: {available_channels}")

            try:
                channel_indices = [available_channels.index(channel) for channel in channels]
            except ValueError as e:
                print(f"Error: {e}. Skipping file {filename}.")
                f._close()
                continue

            extracted_data = [f.readSignal(idx) for idx in channel_indices]
            extracted_data = np.array(extracted_data)

            f._close()

            hdl = pyedflib.EdfWriter(output_filepath, len(channels), f.filetype)
            hdl.setSignalHeaders([f.getSignalHeader(idx) for idx in channel_indices])
            hdl.writeSamples(extracted_data)
            hdl.close()


def main():
    seizure_input_folder = r"SEIZURE COMPLETE EXTRACTION"
    non_seizure_input_folder = r"C:\Users\nithy\Desktop\EEG analysis\NON_SEIZURE_EXTRACTION\NON_SEIZURE_EXTRACTION"
    seizure_output_folder = r"C:\Users\nithy\Desktop\EEG analysis\COMMON_CHANNELS_SEIZURE"
    non_seizure_output_folder = r"C:\Users\nithy\Desktop\EEG analysis\COMMON_CHANNELS_NON_SEIZURE"

    channels = [
        "FP1-F7", "F7-T7", "T7-P7", "P7-O1", "FP1-F3",
        "F3-C3", "C3-P3", "P3-O1", "FZ-CZ", "CZ-PZ",
        "FP2-F4", "F4-C4", "C4-P4", "P4-O2", "FP2-F8",
        "F8-T8", "T8-P8", "P8-O2"
    ]

    extract_channels(seizure_input_folder, seizure_output_folder, channels)
    extract_channels(non_seizure_input_folder, non_seizure_output_folder, channels)


if __name__ == "__main__":
    main()

In [ ]:
import pyedflib
import csv
import os

def extract_channels_from_edf(edf_file_path):
    f = pyedflib.EdfReader(edf_file_path)
    n_channels = f.signals_in_file
    channel_labels = f.getSignalLabels()
    f.close()
    return n_channels, channel_labels

def process_edf_files_in_folder(folder_path, csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Number of Channels", "Channel Names"])

        for filename in os.listdir(folder_path):
            if filename.endswith(".edf"):
                edf_file_path = os.path.join(folder_path, filename)
                n_channels, channel_labels = extract_channels_from_edf(edf_file_path)
                writer.writerow([filename, n_channels, ', '.join(channel_labels)])

    print(f"Processed EDF files in {folder_path}. Channel information saved to {csv_file_path}")
folder_path = r"/content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0"
csv_file_path = r"/content/drive/MyDrive/EEG Signal Classification/channels_info.csv"
process_edf_files_in_folder(folder_path, csv_file_path)


Processed EDF files in /content/drive/MyDrive/EEG DATASET/chb-mit-scalp-eeg-database-1.0.0. Channel information saved to /content/drive/MyDrive/EEG Signal Classification/channels_info.csv


In [ ]:
import os
import pyedflib
import numpy as np


def extract_channels(input_folder, output_folder, channels):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".edf"):
            input_filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, filename)

            f = pyedflib.EdfReader(input_filepath)
            available_channels = f.getSignalLabels()
            print(f"Available channels in {filename}: {available_channels}")

            try:
                channel_indices = [available_channels.index(channel) for channel in channels]
            except ValueError as e:
                print(f"Error: {e}. Skipping file {filename}.")
                f._close()
                continue

            extracted_data = [f.readSignal(idx) for idx in channel_indices]
            extracted_data = np.array(extracted_data)

            f._close()

            hdl = pyedflib.EdfWriter(output_filepath, len(channels), f.filetype)
            hdl.setSignalHeaders([f.getSignalHeader(idx) for idx in channel_indices])
            hdl.writeSamples(extracted_data)
            hdl.close()


def main():
    seizure_input_folder = r"/content/drive/MyDrive/EEG Signal Classification/SEIZURE_EXTRACTION"
    non_seizure_input_folder = r"/content/drive/MyDrive/EEG Signal Classification/NON_SEIZURE_EXTRACTION"
    seizure_output_folder = r"/content/drive/MyDrive/EEG Signal Classification/COMMON_CHANNELS_SEIZURE"
    non_seizure_output_folder = r"/content/drive/MyDrive/EEG Signal Classification/COMMON_CHANNELS_NON_SEIZURE"

    channels = [
        "FP1-F7", "F7-T7", "T7-P7", "P7-O1", "FP1-F3",
        "F3-C3", "C3-P3", "P3-O1", "FZ-CZ", "CZ-PZ",
        "FP2-F4", "F4-C4", "C4-P4", "P4-O2", "FP2-F8",
        "F8-T8", "T8-P8", "P8-O2"
    ]

    extract_channels(seizure_input_folder, seizure_output_folder, channels)
    extract_channels(non_seizure_input_folder, non_seizure_output_folder, channels)


if __name__ == "__main__":
    main()

Available channels in chb10_12_seizure1.edf: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8']
Available channels in chb10_20_seizure1.edf: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8']
Available channels in chb10_27_seizure1.edf: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8']
Available channels in chb10_30_seizure1.edf: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ',

In [ ]:
import os
import numpy as np
import pyedflib

def read_edf_file(file_path):
    with pyedflib.EdfReader(file_path) as f:
        n_channels = f.signals_in_file
        signal_labels = f.getSignalLabels()
        sigbufs = np.zeros((n_channels, f.getNSamples()[0]))

        for i in np.arange(n_channels):
            sigbufs[i, :] = f.readSignal(i)

    return sigbufs

def process_edf_files(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.edf'):
            file_path = os.path.join(input_folder, file_name)
            matrix = read_edf_file(file_path)
            output_file_path = os.path.join(output_folder, file_name.replace('.edf', '.npy'))
            np.save(output_file_path, matrix)

def main(seizure_folder, non_seizure_folder, output_seizure_folder, output_non_seizure_folder):
    process_edf_files(seizure_folder, output_seizure_folder)
    process_edf_files(non_seizure_folder, output_non_seizure_folder)

if __name__ == "__main__":
    seizure_folder = r"/content/drive/MyDrive/EEG Signal Classification/COMMON_CHANNELS_SEIZURE"
    output_seizure_folder = r"/content/drive/MyDrive/EEG Signal Classification/npy_seizures"

    non_seizure_folder = r"/content/drive/MyDrive/EEG Signal Classification/COMMON_CHANNELS_NON_SEIZURE"
    output_non_seizure_folder = r"/content/drive/MyDrive/EEG Signal Classification/npy_non_seizures"
    main(seizure_folder, non_seizure_folder, output_seizure_folder, output_non_seizure_folder)
